<a href="https://colab.research.google.com/github/buster2015/jike_ai_learning/blob/master/homework_0430.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# coding = 'utf-8'
import numpy as np
import pandas as pd
import time

def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result


def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])#申请同等大小的0元素数组
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
            
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [2]:
%reload_ext Cython

In [5]:
%%cython

cimport numpy as np
import numpy as np

cpdef target_mean_v4(long [:,:] data, str y_name, str x_name):
    result = np.zeros(data.shape[0])#申请同等大小的0元素数组
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data[i,1] not in value_dict.keys():
            value_dict[data[i,1]] = data[i,0]
            count_dict[data[i,1]] = 1
        else:
            value_dict[data[i,1]] += data[i,0]
            count_dict[data[i,1]] += 1
    
    for i in range(data.shape[0]):
        result[i] = (value_dict[data[i,1]] - data[i,0]) / (count_dict[data[i,1]] - 1)
    return result

In [6]:
import pandas as pd
import time 
def main():
    y = np.random.randint(2, size=(5000, 1))
    x = np.random.randint(10, size=(5000, 1))
    data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
    
    
    time_start_2 = time.time() #开始计时
    result_2 = target_mean_v2(data, 'y', 'x')
    time_end_2 = time.time() #结束计时
    time_c_2= time_end_2 - time_start_2   #运行所花时间
    print('time cost', time_c_2, 's')
    
    time_start_4 = time.time() #开始计时
    result_4 = target_mean_v4(data.values, 'y', 'x')
    time_end_4 = time.time() #结束计时
    time_c_4= time_end_4 - time_start_4   #运行所花时间
    print('time cost', time_c_4, 's')
    diff = np.linalg.norm(result_4 - result_2)
    print(diff)
    
if __name__ == '__main__':
    main()

time cost 0.3282506465911865 s
time cost 0.0023767948150634766 s
0.0
